# Create Slice for DYNAMOS in FABRIC 

This Jupyter notebook will create a FABRIC slice with resources that can be used for DYNAMOS in FABRIC.

### FABlib API References Examples

FABRIC API docs: https://fabric-fablib.readthedocs.io/en/latest/index.html

Example list of useful functions:
- [fablib.show_config](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.show_config)
- [fablib.list_sites](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.list_sites)
- [fablib.list_hosts](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.list_hosts)
- [fablib.new_slice](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.new_slice)
- [slice.add_node](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.add_node)
- [slice.submit](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.submit)
- [slice.get_nodes](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.get_nodes)
- [slice.list_nodes](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.list_nodesß)
- [slice.show](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.show)
- [node.execute](https://fabric-fablib.readthedocs.io/en/latest/node.html#fabrictestbed_extensions.fablib.node.Node.execute)
- [slice.delete](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.delete)


## Step 1: Configure the Environment (has to be done once in the Jupyter Hub environment)

Before running this notebook, you will need to configure your environment using the [Configure Environment](./configure_and_validate.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook. Note: this has to be done only once in the Jupyter Hub environment (unless configuration is removed/deleted of course).

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

After following all steps of the Configuring Environment notebook, you should be able to run this notebook without additional steps.

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).
 

## Step 2: Setup the Environment for this Notebook

### Step 2.1: Import FABRIC API and other libraries

In [ ]:
import datetime
import traceback
from ipaddress import ip_address, IPv6Address

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();


### (Optional): Query for Available Testbed Resources and Settings

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity and other settings or available resources.

In [ ]:
# Comment out lists you do not want to view

# List available sites
# fablib.list_sites()
# List available hosts
# fablib.list_hosts(includes=["CERN", "AMST"])
# List available images that can be used for the nodes
fablib.get_image_names()

### Step 2.2: Configure the parameters and variables
Can be used to set the corresponding slice and other variables used for subsequent cells.

In [ ]:
# Name of the slice
slice_name = 'DYNAMOS_EnergyEfficiency'
# Set image to ubuntu specific (used in FABRIC Kubernetes cluster example)
image = "default_ubuntu_24"
# Site used for the nodes (Use the UvA site: AMST)
site = "AMST"
# Host of the site used for the nodes (Use specific host to avoid error occurred without it: "Timeout waiting for the server to come up")
site_host = "amst-w1.fabric-testbed.net"
# Nodes:
node1_name = 'k8s-control-plane'
node2_name = 'dynamos-core'
node3_name = 'vu'
node4_name = 'uva'
node5_name = 'surf'
# Node specifications (make it not too low for all nodes, otherwise it will be very slow)
node_disk = 100
control_plane_ram = 16
control_plane_cores = 4
# Worker should have more than control plane node
worker_ram = 16
worker_cores = 8
# Network:
node1_nic_name = 'NIC1'
node2_nic_name = 'NIC2'
node3_nic_name = 'NIC3'
node4_nic_name = 'NIC4'
node5_nic_name = 'NIC5'
network_name = 'NET1'

## Step 3: Create the Slice and Nodes

The following creates a slice with the required nodes. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated. 

By default, the submit function will block until the node is ready and will display the progress of your slice being built (it will automatically retry until the slice is submitted (i.e. the state of the slice is not configuring anymore)).

Note: the slice must incorporate resources to be created, it cannot be created with no resources here, giving the error: "Exception: Submit request error: return_status Status.INVALID_ARGUMENTS, slice_reservations: Either topology None or slice graph None must be specified".

### This includes: Setup Access to IPv4 from IPv6 only FABRIC VMs
See fabric/dynamos/Troubleshooting.md for more explanation about this related issue. In short, the FABRIC VMs have IPv6 access only as their management IP (i.e. main IP). However, not all platforms support IPv6. This is most important for some custom registries outside the big ones like Github, such as "cr.l5d.io", which caused some problems before. Also, this might fix some other connectivity issues that occured.



In [ ]:
try:
    # =================================================== Step 1: Create the Slice (including configuring network) ===================================================
    print("=================================================== Creating slice... =================================================== ")
    # Create a slice
    slice = fablib.new_slice(name=slice_name)

    # Add Nodes with the specific variables
    # Also validate the node can be created and raise an exception in case of failure
    # Control plane node:
    node1 = slice.add_node(name=node1_name, site=site, validate=True, raise_exception=True, host=site_host, image=image, cores=control_plane_cores, ram=control_plane_ram, disk=node_disk)
    # Worker nodes:
    node2 = slice.add_node(name=node2_name, site=site, validate=True, raise_exception=True, host=site_host, image=image, cores=worker_cores, ram=worker_ram, disk=node_disk)
    node3 = slice.add_node(name=node3_name, site=site, validate=True, raise_exception=True, host=site_host, image=image, cores=worker_cores, ram=worker_ram, disk=node_disk)
    node4 = slice.add_node(name=node4_name, site=site, validate=True, raise_exception=True, host=site_host, image=image, cores=worker_cores, ram=worker_ram, disk=node_disk)
    node5 = slice.add_node(name=node5_name, site=site, validate=True, raise_exception=True, host=site_host, image=image, cores=worker_cores, ram=worker_ram, disk=node_disk)

    # Add network interface components: https://learn.fabric-testbed.net/knowledge-base/glossary/#component
    # https://learn.fabric-testbed.net/knowledge-base/network-interfaces-in-fabric-vms/
    # https://fabric-fablib.readthedocs.io/en/latest/node.html#fabrictestbed_extensions.fablib.node.Node.add_component
    iface1 = node1.add_component(model='NIC_Basic', name=node1_nic_name).get_interfaces()[0]
    iface2 = node2.add_component(model='NIC_Basic', name=node2_nic_name).get_interfaces()[0]
    iface3 = node3.add_component(model='NIC_Basic', name=node3_nic_name).get_interfaces()[0]
    iface4 = node4.add_component(model='NIC_Basic', name=node4_nic_name).get_interfaces()[0]
    iface5 = node5.add_component(model='NIC_Basic', name=node5_nic_name).get_interfaces()[0]
    # Set to auto mode to let FABlib allocate an IP (=ip_addr_add) from the network's subnet and configure the device during the post boot configuration stage
    # See https://github.com/fabric-testbed/jupyter-examples/blob/main/fabric_examples/fablib_api/create_l3network_fabnet_ipv4/create_l3network_fabnet_ipv4_auto.ipynb
    iface1.set_mode('auto')
    iface2.set_mode('auto')
    iface3.set_mode('auto')
    iface4.set_mode('auto')
    iface5.set_mode('auto')
    # Add network with IPv4 (better for K8s than IPv6 currently): https://learn.fabric-testbed.net/knowledge-base/network-services-in-fabric/
    # One network is enough, since all nodes are on the same site
    net1 = slice.add_l3network(name=network_name, interfaces=[iface1, iface2, iface3, iface4, iface5], type='IPv4')
    # Add routes for the interfaces to the nodes
    node1.add_route(subnet=fablib.FABNETV4_SUBNET, next_hop=net1.get_gateway())
    node2.add_route(subnet=fablib.FABNETV4_SUBNET, next_hop=net1.get_gateway())
    node3.add_route(subnet=fablib.FABNETV4_SUBNET, next_hop=net1.get_gateway())
    node4.add_route(subnet=fablib.FABNETV4_SUBNET, next_hop=net1.get_gateway())
    node5.add_route(subnet=fablib.FABNETV4_SUBNET, next_hop=net1.get_gateway())

    # Calculate the lease end time for 2 weeks from now with timezone information
    lease_end_time = datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(weeks=2)

    # Submit the slice request, using an end date 2 weeks from now (the current maximum lease time) 
    # to make sure that the slice can be used for a longer period of time. Progress shows an indicator of the current progression.
    # Wait until the state is finished and use an interval (it may take some time before the slice and nodes are created)
    slice.submit(wait=True, wait_timeout=3600, wait_interval=20, progress=True, wait_jupyter='text', lease_end_time=lease_end_time)

    # =================================================== Step 2: Configure Nodes (e.g. setup IPv4 access) ===================================================
    print("=================================================== Setting IPv4-only resources access... =================================================== ")
    # enable nodes to access IPv4-only resources, such as Github, even if the control interface is IPv6-only
    # This is an addition on what is already there, as Nat64 is already configured by default now, see: https://learn.fabric-testbed.net/knowledge-base/using-ipv4-only-resources-like-github-or-docker-hub-from-ipv6-fabric-sites/#reaching-ipv4-resources-via-nat64
    # See for this guide that is an addition: https://github.com/teaching-on-testbeds/fabric-snippets/blob/main/ipv4-access-from-ipv6.md
    for node in slice.get_nodes():
        # Print before editing the files:
        print(f"==================== Files before configuring for node: {node.get_name()} ====================")
        node.execute('cat /etc/resolv.conf')
        node.execute('cat /etc/hosts')
        # Check if the node has IPv6 management IP
        if type(ip_address(node.get_management_ip())) is IPv6Address:
            # Adds a working public IPv6 DNS server (from SURFnet/AMS-IX) to the systemd-resolved config file:
            # 2a00:1098:2c::1 is a known reliable IPv6 DNS resolver.
            node.execute('echo "DNS=2a00:1098:2c::1" | sudo tee -a /etc/systemd/resolved.conf')
            # Restarts the DNS resolver service to apply the new config
            node.execute('sudo service systemd-resolved restart')
            # Adds a loopback entry for the hostname to /etc/hosts
            # Fixes issues where hostname -s cannot resolve to localhost
            # Common workaround for tools like sudo, ssh, or certain init systems that rely on hostname resolution
            node.execute('echo "127.0.0.1 $(hostname -s)" | sudo tee -a /etc/hosts')
            # Replaces /etc/resolv.conf with the systemd-managed symlink
            # On many systems, /etc/resolv.conf is incorrectly set (or overwritten by cloud-init). This command:
            # Deletes any existing file, and recreates it as a symlink to the correct DNS config managed by systemd-resolved
            node.execute('sudo rm -f /etc/resolv.conf; sudo ln -sv /run/systemd/resolve/resolv.conf /etc/resolv.conf')

        # Print after editing the files:
        print(f"==================== Files after configuring for node: {node.get_name()} ====================")
        node.execute('cat /etc/resolv.conf')
        node.execute('cat /etc/hosts')

    # =================================================== Step 3: Verify Network ===================================================
    print("=================================================== Verifying network setup... =================================================== ")
    # ========== Step 3.1: Get Slice and Network ==========
    # Get the network
    network = slice.get_network(name=network_name)
    # Print network
    print(f"{network}")

    # ========== Step 3.2: Check connectivity for each node ==========
    for node in slice.get_nodes():
        print(f"==================== Verifying for node: {node.get_name()} ====================")
        # Get interface
        node_iface = node.get_interface(network_name=network_name)
        # Get ip address of the node
        node_addr = node_iface.get_ip_addr()
        
        # Print the IP to be used with kubernetes
        print(f"IPv4 from interface {node_iface} for {node.get_name()}: {node_addr}")

        # Connectivity tests
        print(f"Testing connectivity to outside world... ")
        stdout, stderr = node.execute(f'ping -c 5 google.com')
        print("Testing connectivity to all nodes... ")
        # Test connectivity for each node (including itself)
        for temp_node in slice.get_nodes():
            print(f"Testing connectivity to node {temp_node.get_name()}... ")
            # Get IP of this node and check with ping if it can reach it
            temp_node_addr = temp_node.get_interface(network_name=network_name).get_ip_addr()
            stdout, stderr = node.execute(f'ping -c 5 {temp_node_addr}')

        # Interface IP info and routing table information
        print("Listing IP routes and addresses... ")
        stdout, stderr = node.execute(f'ip route list')
        stdout, stderr = node.execute(f'ip addr show {node_iface.get_os_interface()}')

except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

## Step 3: Observe the Slice's Attributes
This step just observes the slice's attributes. 

### Show the slice attributes 

In [ ]:
slice.show();

#### List the nodes

In [ ]:
slice.list_nodes();

### Run a test Experiment to Validate the nodes

Most experiments will require automated configuration and execution. You can use the fablib library to execute arbitrary commands on your node. 

The following code demonstrates how to use fablib to execute a "Hello, FABRIC" bash script. The library uses the bastion and VM keys defined at the top of this notebook to jump through the bastion host and execute the script.

In [ ]:
#node = slice.get_node('Node1')

for node in slice.get_nodes():
    stdout, stderr = node.execute('echo Hello, FABRIC from node `hostname -s`')